# ex 1


In [3]:
import numpy as np
# from np import 

A = np.array([[2,3,4,5], [0,1,1,3],[1,2,3,4],[1,1,2,2]])

print(np.linalg.det(A))

1.0


In [4]:
'''=============================================================================
Minh họa: Matrix --> Compressed Sparse Row (CSR format): row oriented
   (Xem ví dụ bên dưới)
   Giả sử ma trận A có m dòng và A có p phần từ <> 0
   
   ** Tạo 3 arrays có kích thước khác nhau:
      1) data[p]      : chứa các giá trị <> 0
      2) indices[p]   : chứa các chỉ số CỘT của các phần tử <> 0
      3) indptr[m + 1]: chứa một dãy số tăng (không đều) từ 0 cho đến p
                        là VỊ TRÍ BẮT ĐẦU rút trích trong data[] cho mỗi dòng
         idxptr = (start_r(0), start_r(1), start_r(2), ..., start_r(m-1), p)
         
         Lưu ý: indptr[0]     = start_r(0) = 0
                idxptr[m + 1] = p
                
         VD: idxptr = (0, 2, 5, 7, 9, 12)
         
         Gọi P(i) là tập các giá trị <> 0 của dòng i, rút trích từ trong data[]
         P(0) = { data[0], data[1] }
         P(1) = { data[2], data[3], data[4] }
         P(2) = { data[5], data[6] }
         P(3) = { data[7], data[8] }
         P(4) = { data[9], data[10], data[11] }
============================================================================='''
import numpy as np
import scipy.sparse as sparse

A = np.array([[8, 0, 0, 6, 0, 0], 
              [0, 0, 9, 0, 0, 4],
              [0, 0, 0, 3, 0, 0]])

# Matrix --> CSR format    
csr = sparse.csr_matrix(A)

# DATA array:     [8 6 9 4 3]
print('DATA    array: ', csr.data)

# INDICES array:  [0 3 2 5 3]
print('INDICES array: ', csr.indices)

''' Các bước tạo nên mảng INDPTR[] có (m + 1 = 4) phần tử (ma trận A có 3 dòng)
      * INDPTR[] = [0 ? ? 5] vì A có p = 5 phần tử <> 0 và INDPTR[0] luôn luôn = 0

      * Dòng i = 1 có phần tử <> 0 đầu tiên là 9, tương ứng với phần tử data[2]
                   cho nên idxptr[i = 1] = 2
                    --> INDPTR[] = [0 2 ? 5]
      * Dòng i = 2 có phần tử <> 0 đầu tiên là 3, tương ứng với phần tử data[4]
                   cho nên idxptr[i = 2] = 4
                    --> INDPTR[] = [0 2 4 5]
'''
print('INDPTR  array: ', csr.indptr)


'''----------------------------------------------------------------------------
Minh họa: CSR format --> matrix
----------------------------------------------------------------------------'''
d   = np.array([8, 6, 9, 4, 3])
ind = np.array([0, 3, 2, 5, 3])
ptr = np.array([0, 2, 4, 5])
mtx = sparse.csr_matrix((d, ind, ptr))
print('\nTái tạo ma trận từ CSR format (default shape):\n', mtx.todense())
'''   * ptr[] có 4 phần tử ==> ma trận có 3 dòng
      * ptr[0] = 0
        ptr[1] = 2
        ptr[2] = 4
        ptr[3] = 5 ==> ma trận có 5 phần tử khác 0 lưu trữ trong d[]
        
      * P(0): lấy từ chỉ số ptr[0] = 0, nghĩa là từ d[0], cho đến d[1]
              vì d[2] là chỉ số bắt đầu của P(1)
              P(0) = {d[0], d[1]} = {8, 6}
                 d[0] --> indices[0] = 0 --> A(0, 0) = 8
                 d[1] --> indices[1] = 3 --> A(0, 3) = 6
              
      * P(1): lấy từ chỉ số ptr[1] = 2, nghĩa là từ d[2], cho đến d[3]
              vì d[4] là chỉ số bắt đầu của P(2)
              P(1) = {d[2], d[3]} = {9, 4}
                 d[2] --> indices[2] = 2 --> A(1, 2) = 9
                 d[3] --> indices[3] = 5 --> A(1, 5) = 4
              
      * P(2): lấy từ chỉ số ptr[2] = 4, nghĩa là từ d[4], cho đến hết
              vì là dòng cuối cùng của ma trận
                 d[4] --> indices[4] = 3 --> A(2, 3) = 3

        [8 0 0 6 0 0]
        [0 0 9 0 0 4]
        [0 0 0 3 0 0]
'''

mtx = sparse.csr_matrix((d, ind, ptr), shape = (3, 9))
print('\nTái tạo ma trận từ CSR format (explicit shape):\n', mtx.todense())





DATA    array:  [8 6 9 4 3]
INDICES array:  [0 3 2 5 3]
INDPTR  array:  [0 2 4 5]

Tái tạo ma trận từ CSR format (default shape):
 [[8 0 0 6 0 0]
 [0 0 9 0 0 4]
 [0 0 0 3 0 0]]

Tái tạo ma trận từ CSR format (explicit shape):
 [[8 0 0 6 0 0 0 0 0]
 [0 0 9 0 0 4 0 0 0]
 [0 0 0 3 0 0 0 0 0]]


In [5]:
"""=============================================================================
Ex1: DECOMPOSITION
    Câu 1: LU Decomposition
        a) Tạo ma trận A(4 x 4) chứa các giá trị ngẫu nhiên trong khoảng 1 - 10
        b) Phân tích thành các thành phần P, L, U. In kết quả
        c) Tái tạo lại ma trận A từ các thành phần P, L, U
============================================================================="""
    
import numpy  as np
import random       
from scipy.linalg import lu

##------------------------------------------------------------------------------
## Hàm tạo 1 ma trận A[mxn] với giá trị ngẫu nhiên thuộc [start, end]
##------------------------------------------------------------------------------
def create_matrix_random(m, n, start, end):
    mtr = []
    for i in range(m):
        row = []
        for j in range(n):
            a = random.randint(start, end + 1)
            
            # Thêm giá trị vào dòng hiện hành 
            row.append(a)
            
        # Thêm dòng vào ma trận    
        mtr.append(row)
        
    return np.array(mtr)
##------------------------------------------------------------------------------

## a) Tạo ma trận A(4 x 4) chứa các giá trị ngẫu nhiên trong khoảng 1 - 10
## Ma trận HÌNH CHỮ NHẬT: A(m, n) = P_T(m, m).L(m, n).U(n, n)
m, n, min, max = 4, 4, 1, 10
A = create_matrix_random(m, n, min, max)
print('Ma trận VUÔNG A', A.shape, ':\n', A)

## b) Phân tích thành các thành phần P, L, U. In kết quả
print('\n*** Áp dụng phân rã PLU:')
P_T, L, U = lu(A)

print('Ma trận P_T', P_T.shape, ':\n', P_T)
print('Ma trận L', L.shape, ':\n', L)
print('Ma trận U', U.shape, ':\n', U)

print('\nTái tạo A từ P, L, U (kiểm chứng lại phép phân rã):\n', P_T.dot(L).dot(U), '\n')

##------------------------------------------------------------------------------
## Mở rộng cho ma trận hình chữ nhật
##------------------------------------------------------------------------------
A = np.array([[1, 2,  4, 3], 
              [3, 8, 14, 0],
              [2, 6, 13, 2]])
P_T, L, U = lu(A)

print('Ma trận HÌNH CHỮ NHẬT A\n', A.shape, A)
print('\n*** Áp dụng phân rã PLU')
print('Ma trận P_T', P_T.shape, ':\n', P_T)
print('Ma trận L', L.shape, ':\n', L)
print('Ma trận U', U.shape, ':\n', U)
print('\nTái tạo A từ P, L, U (kiểm chứng lại phép phân rã):\n', P_T.dot(L).dot(U), '\n')




Ma trận VUÔNG A (4, 4) :
 [[ 8  3 10  9]
 [ 8  1  4  8]
 [ 5  8 11 11]
 [11  2 10  2]]

*** Áp dụng phân rã PLU:
Ma trận P_T (4, 4) :
 [[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
Ma trận L (4, 4) :
 [[ 1.          0.          0.          0.        ]
 [ 0.45454545  1.          0.          0.        ]
 [ 0.72727273 -0.06410256  1.          0.        ]
 [ 0.72727273  0.21794872 -0.46188341  1.        ]]
Ma trận U (4, 4) :
 [[11.          2.         10.          2.        ]
 [ 0.          7.09090909  6.45454545 10.09090909]
 [ 0.          0.         -2.85897436  7.19230769]
 [ 0.          0.          0.          8.66816143]]

Tái tạo A từ P, L, U (kiểm chứng lại phép phân rã):
 [[ 8.  3. 10.  9.]
 [ 8.  1.  4.  8.]
 [ 5.  8. 11. 11.]
 [11.  2. 10.  2.]] 

Ma trận HÌNH CHỮ NHẬT A
 (3, 4) [[ 1  2  4  3]
 [ 3  8 14  0]
 [ 2  6 13  2]]

*** Áp dụng phân rã PLU
Ma trận P_T (3, 3) :
 [[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]
Ma trận L (3, 3) :
 [[ 1.          0.          0.        ]


In [6]:
"""=============================================================================
Ex1: DECOMPOSITION
    Câu 2: Giải hệ phương trình bằng LU Decomposition
        a) Quy về Ax = B. Sau đó giải tìm x.
        b) Phân tích thành các thành phần P, L, U. In kết quả.
        c) Tái tạo lại ma trận A từ các thành phần P, L, U.
============================================================================="""
import numpy        as np    
import scipy.linalg as linalg

from scipy.linalg import lu

## Câu 2
A = np.array([[2, 1, 1], 
              [1, 3, 2], 
              [1, 0, 0]])
B = np.array([4, 5, 6]) 

## Ví dụ thêm
A = np.array([[10, 8, 10,  8], 
              [ 8, 6,  6, 10], 
              [ 3, 3,  6,  3],
              [11, 8,  6,  1]])
B = np.array([4, 5, 6, 7]) 

print('Ma trận A', A.shape, ':\n', A)

print('\n************************ Hàm scipy.linalg.lu() ***********************')
P_T, L, U = lu(A)

print('Ma trận P_T', P_T.shape, ':\n', P_T, '\n')
print('Ma trận L', L.shape, ':\n', L, '\n')
print('Ma trận U', U.shape, ':\n', U, '\n')


print('\n********************* Hàm scipy.linalg.lu_factor() *******************')
## //docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.linalg.lu_factor.html
##    Hàm lu_factor(A) trả về 2 đối tượng:
##       - Ma trận nxn: U + (L - diag(L))
##       - Mảng [n] chứa các chỉ số dòng của I(n) để tạo nên P
##------------------------------------------------------------------------------
## Hàm tạo ma trận hoán vị P từ pivot = linalg.lu_factor(A)[1]
def permutationMatrix(pivot):
    # Khởi tạo P bằng ma trận đơn vị
    P = np.identity(len(pivot))

    # i: index; r: element
    for i, r in enumerate(pivot):
        # pivot[i] = r
        # Hoán đổi dòng r và dòng pivot[i] của ma trận I
    
        I = np.identity(len(pivot))
    
        temp    = I[i, :].copy()
        I[i, :] = I[r, :]
        I[r, :] = temp 
        
        P = P.dot(I)
    return P
##------------------------------------------------------------------------------

LU = linalg.lu_factor(A)

print('Ma trận kết hợp U + L - diag(L)', LU[0].shape, ':\n', LU[0])

U = np.triu(LU[0])
L = LU[0] - U + np.identity(len(B))

print('Ma trận L', L.shape, ':\n', L, '\n')
print('Ma trận U', U.shape, ':\n', U, '\n')

## Ma trận hoán vị
print('Mảng pivot', LU[1].shape, ':\n', LU[1])
P = permutationMatrix(LU[1])
print('Ma trận hoán vị P', P.shape, ':\n', P, '\n')

x = linalg.lu_solve(LU, B)
print("Hệ nghiệm [xi] =", x)


Ma trận A (4, 4) :
 [[10  8 10  8]
 [ 8  6  6 10]
 [ 3  3  6  3]
 [11  8  6  1]]

************************ Hàm scipy.linalg.lu() ***********************
Ma trận P_T (4, 4) :
 [[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]] 

Ma trận L (4, 4) :
 [[1.         0.         0.         0.        ]
 [0.27272727 1.         0.         0.        ]
 [0.90909091 0.88888889 1.         0.        ]
 [0.72727273 0.22222222 1.         1.        ]] 

Ma trận U (4, 4) :
 [[11.          8.          6.          1.        ]
 [ 0.          0.81818182  4.36363636  2.72727273]
 [ 0.          0.          0.66666667  4.66666667]
 [ 0.          0.          0.          4.        ]] 


********************* Hàm scipy.linalg.lu_factor() *******************
Ma trận kết hợp U + L - diag(L) (4, 4) :
 [[11.          8.          6.          1.        ]
 [ 0.27272727  0.81818182  4.36363636  2.72727273]
 [ 0.90909091  0.88888889  0.66666667  4.66666667]
 [ 0.72727273  0.22222222  1.          4.        ]]
Ma tr

In [7]:
"""=============================================================================
Ex1: DECOMPOSITION
    Câu 3: QA Decomposition
        a) Tạo ma trận A(4 x 4) chứa các giá trị ngẫu nhiên trong khoảng 5 - 10
        b) Phân tích thành các thành phần Q và R. In kết quả
        c) Tái tạo lại ma trận A từ các thành phần Q và R
============================================================================="""
    
import numpy  as np
import random       
from scipy.linalg import qr

##------------------------------------------------------------------------------
## Hàm tạo 1 ma trận A[mxn] với giá trị ngẫu nhiên thuộc [start, end]
##------------------------------------------------------------------------------
def create_matrix_random(m, n, start, end):
    mtr = []
    for i in range(m):
        row = []
        for j in range(n):
            a = random.randint(start, end + 1)
            
            # Thêm giá trị vào dòng hiện hành 
            row.append(a)
            
        # Thêm dòng vào ma trận    
        mtr.append(row)
        
    return np.array(mtr)
##------------------------------------------------------------------------------

## a) Tạo ma trận A(4 x 4) chứa các giá trị ngẫu nhiên trong khoảng 5 - 10
m, n, min, max = 4, 4, 5, 10
A = create_matrix_random(m, n, min, max)
print('Ma trận A', A.shape, ':\n', A)

## b) Phân tích thành các thành phần Q và R. In kết quả
print('\n*** Áp dụng phân rã QR:')
Q, R = qr(A)

print('Ma trận Q', Q.shape, ':\n', Q)
print('Ma trận R', R.shape, ':\n', R)

## c) Tái tạo lại ma trận A từ các thành phần Q và R
print('\nTái tạo A từ Q, R (kiểm chứng lại phép phân rã):\n', Q.dot(R))


Ma trận A (4, 4) :
 [[ 8  5  8  7]
 [ 7 11  7  5]
 [10  9  7 10]
 [ 6  9  5  7]]

*** Áp dụng phân rã QR:
Ma trận Q (4, 4) :
 [[-0.5069794   0.57704828  0.61171914  0.18917417]
 [-0.44360698 -0.62427999  0.38135655 -0.51773982]
 [-0.63372425  0.25258697 -0.68189261 -0.26384818]
 [-0.38023455 -0.46204934 -0.1240538   0.79154454]]
Ma trận R (4, 4) :
 [[-15.77973384 -16.54020294 -13.49832654 -14.76577504]
 [  0.          -5.86699981  -0.29571158   0.20946237]
 [  0.           0.           2.16973164  -1.49848604]
 [  0.           0.           0.           1.63785001]]

Tái tạo A từ Q, R (kiểm chứng lại phép phân rã):
 [[ 8.  5.  8.  7.]
 [ 7. 11.  7.  5.]
 [10.  9.  7. 10.]
 [ 6.  9.  5.  7.]]


In [8]:
"""=============================================================================
Ex1: DECOMPOSITION
    Câu 4: Cholesky Decomposition
        a) Tạo ma trận A(3 x 3) chứa các giá trị ngẫu nhiên trong khoảng 3 - 9
        b) Chuyển A thành ma trận vuông đối xứng theo tam giác dưới
        c) Thực hiện phân rã Cholesky, nếu có thể
============================================================================="""

import numpy  as np
import random       
from scipy.linalg import cholesky

##------------------------------------------------------------------------------
## Hàm tạo 1 ma trận A[mxn] với giá trị ngẫu nhiên thuộc [start, end]
##------------------------------------------------------------------------------
def create_matrix_random(m, n, start, end):
    mtr = []
    for i in range(m):
        row = []
        for j in range(n):
            a = random.randint(start, end + 1)
            
            # Thêm giá trị vào dòng hiện hành 
            row.append(a)
            
        # Thêm dòng vào ma trận    
        mtr.append(row)
        
    return np.array(mtr)
##------------------------------------------------------------------------------

## a) Tạo ma trận A(3 x 3) chứa các giá trị ngẫu nhiên trong khoảng 3 - 9
m, n, min, max = 3, 3, 3, 9
A = create_matrix_random(m, n, min, max)
print('*** Ma trận A', A.shape, ':\n', A)

## b) Chuyển A thành ma trận vuông đối xứng theo tam giác dưới
## Điều kiện phân rã Cholesky: ma trận ĐỐI XỨNG
for i in range(A.shape[0]):
    for j in range(i):
        A[j][i] = A[i][j]

## c) Thực hiện phân rã Cholesky, nếu có thể
## Kiểm tra ma trận xác định dương
test = np.linalg.eigvalsh(A)

pos_def = np.all(test > 0)   
if (pos_def == True):
    L = cholesky(A)

    print('Ma trận L', L.shape, ':\n', L)

    print('Tái tạo ma trận A:\n', L.dot(L.T))
else:
    print('\nMa trận A không thỏa điều kiện XÁC ĐỊNH DƯƠNG')
    
##------------------------------------------------------------------------------
## Hàm tạo ma trận xác định dương    
##------------------------------------------------------------------------------
def create_matrix_positive_definite(m, n, start, end):
    A       = None
    pos_def = False
    
    while (pos_def == False):
        A = create_matrix_random(m, n, start, end)
        for i in range(A.shape[0]):
            for j in range(i):
                A[j][i] = A[i][j]
        test    = np.linalg.eigvalsh(A)
        pos_def = np.all(test > 0)
    return A
##------------------------------------------------------------------------------
A = create_matrix_positive_definite(3, 3, 3, 9)
print('\n*** Ma trận A', A.shape, 'thỏa điều kiện XÁC ĐỊNH DƯƠNG:\n', A)

print('Áp dụng phân rã Cholesky:')
L = cholesky(A)
print('Ma trận L', L.shape, ':\n', L)

print('\nTái tạo A từ L, L_T (kiểm chứng lại phép phân rã)\n', L.dot(L.T))

*** Ma trận A (3, 3) :
 [[10  9  3]
 [ 8  9  3]
 [ 7  7  9]]
Ma trận L (3, 3) :
 [[3.16227766 2.52982213 2.21359436]
 [0.         1.61245155 0.86824314]
 [0.         0.         1.82924953]]
Tái tạo ma trận A:
 [[21.3         6.00115374  4.04921645]
 [ 6.00115374  3.35384615  1.58823336]
 [ 4.04921645  1.58823336  3.34615385]]

*** Ma trận A (3, 3) thỏa điều kiện XÁC ĐỊNH DƯƠNG:
 [[10  5  6]
 [ 5  6  7]
 [ 6  7 10]]
Áp dụng phân rã Cholesky:
Ma trận L (3, 3) :
 [[3.16227766 1.58113883 1.8973666 ]
 [0.         1.87082869 2.13808994]
 [0.         0.         1.35224681]]

Tái tạo A từ L, L_T (kiểm chứng lại phép phân rã)
 [[16.1         7.01478031  2.56570792]
 [ 7.01478031  8.07142857  2.89122529]
 [ 2.56570792  2.89122529  1.82857143]]


In [9]:
'''=============================================================================
   a) Slide #14 
      A = L.LT = UT.U
         L: ma trận tam giác DƯỚI, khả nghịch, Lii > 0
         U: ma trận tam giác TRÊN, khả nghịch, Uii > 0
    b) Kiểm tra tính chất POSITIVE DEFINITE, POSITIVE SEMIDEFINITE
============================================================================='''
import numpy as np

from numpy        import linalg   as la
from scipy.linalg import cholesky

# from numpy.linalg import cholesky: Default --> LOWER


##------------------------------------------------------------------------------
## Positive definite (SYMETRIC) matrix
##------------------------------------------------------------------------------
A = np.array([[6, 1, 1], 
              [1, 6, 1], 
              [1, 1, 6]])
print('\n*** Ma trận ĐỐI XỨNG:\n', A)

print('Áp dụng phân rã Cholesky:')

## Cholesky decomposition: Default --> UPPER (U)
U = cholesky(A)
print('\nUPPER   Cholesky factor:\n', U)
print('\nTái tạo A từ U, U_T (kiểm chứng lại phép phân rã)\n', U.T.dot(U))

U = cholesky(A, lower = False)
print('\nUPPER   Cholesky factor:\n', U)

L = cholesky(A, lower = True)
print('\nLOWER   Cholesky factor:\n', L)
print('\nTái tạo A từ L, L_T (kiểm chứng lại phép phân rã)\n', L.dot(L.T))


##------------------------------------------------------------------------------
## Positive definite (NON-SYMETRIC) matrix
##------------------------------------------------------------------------------
A = np.array([[  4,  12, -15], 
              [ 12,  37, -42], 
              [-16, -43,  98]])

print('\n*** Ma trận KHÔNG ĐỐI XỨNG:\n', A)

# Cholesky decomposition
print('Áp dụng phân rã Cholesky:')
U = cholesky(A)
print('UPPER   Cholesky factor:\n', U)
print('\nTái tạo A từ U, U_T (kiểm chứng lại phép phân rã)\n', U.T.dot(U))

U = cholesky(A, lower = False)
print('UPPER   Cholesky factor:\n', U)

L = cholesky(A, lower = True)
print('LOWER   Cholesky factor:\n', L)
print('\nTái tạo A từ L, L_T (kiểm chứng lại phép phân rã)\n', L.dot(L.T))


##------------------------------------------------------------------------------
##   Kiểm tra tính chất POSITIVE DEFINITE của ma trận A
##------------------------------------------------------------------------------
# Positive DEFINITE (symmetric) matrix
A = np.array([[9, 6], 
              [6, 5]])

# Eigenvalues
eigenValues, eigenVectors = la.eig(A)
pos_def = np.all(eigenValues > 0)
if (pos_def == True):
    print('\n*** Ma trận', A, '\nlà ma trận xác định dương.')
    print('Áp dụng phân rã Cholesky:')
    print('UPPER Cholesky factor:\n', cholesky(A))
    print('UPPER Cholesky factor:\n', cholesky(A, check_finite = True))
else:
    print('\n*** Ma trận', A, '\nKHÔNG phải là ma trận xác định dương.')
    
# Positive SEMIDEFINITE (symmetric) matrix
#    A positive semidefinite matrix is a Hermitian matrix all of whose eigenvalues
#    are nonnegative.
A = np.array([[9, 6], [6, 4]])

# Eigenvalues
eigenValues, eigenVectors = la.eig(A)
pos_def = np.all(eigenValues > 0)
if (pos_def == True):
    print('\n*** Ma trận', A, '\nlà ma trận xác định dương.')
    print('Áp dụng phân rã Cholesky:')
    print('UPPER Cholesky factor:\n', cholesky(A, check_finite = True))
else:
    print('\n*** Ma trận', A, '\nKHÔNG phải là ma trận xác định dương.')
    
# NOT Positive SEMIDEFINITE (symmetric) matrix
A = np.array([[9, 6], [6, 3]])

# Eigenvalues
eigenValues, eigenVectors = la.eig(A)
pos_def = np.all(eigenValues > 0)
if (pos_def == True):
    print('\n*** Ma trận', A, '\nlà ma trận xác định dương.')
    print('Áp dụng phân rã Cholesky:')
    print('UPPER Cholesky factor:\n', cholesky(A, check_finite = True))
else:
    print('\n*** Ma trận', A, '\nKHÔNG phải là ma trận xác định dương.')



*** Ma trận ĐỐI XỨNG:
 [[6 1 1]
 [1 6 1]
 [1 1 6]]
Áp dụng phân rã Cholesky:

UPPER   Cholesky factor:
 [[2.44948974 0.40824829 0.40824829]
 [0.         2.41522946 0.34503278]
 [0.         0.         2.39045722]]

Tái tạo A từ U, U_T (kiểm chứng lại phép phân rã)
 [[6. 1. 1.]
 [1. 6. 1.]
 [1. 1. 6.]]

UPPER   Cholesky factor:
 [[2.44948974 0.40824829 0.40824829]
 [0.         2.41522946 0.34503278]
 [0.         0.         2.39045722]]

LOWER   Cholesky factor:
 [[2.44948974 0.         0.        ]
 [0.40824829 2.41522946 0.        ]
 [0.40824829 0.34503278 2.39045722]]

Tái tạo A từ L, L_T (kiểm chứng lại phép phân rã)
 [[6. 1. 1.]
 [1. 6. 1.]
 [1. 1. 6.]]

*** Ma trận KHÔNG ĐỐI XỨNG:
 [[  4  12 -15]
 [ 12  37 -42]
 [-16 -43  98]]
Áp dụng phân rã Cholesky:
UPPER   Cholesky factor:
 [[ 2.          6.         -7.5       ]
 [ 0.          1.          3.        ]
 [ 0.          0.          5.72276157]]

Tái tạo A từ U, U_T (kiểm chứng lại phép phân rã)
 [[  4.  12. -15.]
 [ 12.  37. -42.]
 [

In [18]:
"""=============================================================================
Ex2: EIGENDECOMPOSITION
    Câu 1:
        a) Tạo ma trận A(5 x 5) chứa các giá trị ngẫu nhiên trong khoảng 1 - 10
        b) Phân tích eigenvalues và eigenvectors
        c) Kiểm tra eigenvector đầu tiên theo dot và theo eigenvalue có bằng nhau?
           Nếu bằng nhau thì tái tạo A từ các eigenvalues và eigenvectors
============================================================================="""
    
import numpy  as np
import random       

from numpy.linalg import eig, inv
from numpy import diag

##------------------------------------------------------------------------------
## Hàm tạo 1 ma trận A[mxn] với giá trị ngẫu nhiên thuộc [start, end]
##------------------------------------------------------------------------------
def create_matrix_random(m, n, start, end):
    mtr = []
    for i in range(m):
        row = []
        for j in range(n):
            a = random.randint(start, end + 1)
            
            # Thêm giá trị vào dòng hiện hành 
            row.append(a)
            
        # Thêm dòng vào ma trận    
        mtr.append(row)
        
    return np.array(mtr)
##------------------------------------------------------------------------------
    
## a) Tạo ma trận A(5 x 5) chứa các giá trị ngẫu nhiên trong khoảng 1 - 10
m, n, min, max = 5, 5, 1, 10
A = create_matrix_random(m, n, min, max)
print('Ma trận A', A.shape, ':\n', A)




Ma trận A (5, 5) :
 [[ 7 11  1  9  3]
 [11  4  7  3  1]
 [11 10  7  6  8]
 [10  1  8  7  5]
 [10 11  5  4  8]]


In [19]:
## b) Phân tích eigenvalues và eigenvectors
values, vectors = eig(A)
print('\nSố lượng giá trị riêng: %d' %len(values))
print('\nEigenvalues',  values.shape,  ':\n', values)
print('\nEigenvectors', vectors.shape, ':\n', vectors)



Số lượng giá trị riêng: 5

Eigenvalues (5,) :
 [32.55629203+0.j         -7.55852249+0.j          3.61386763+3.79231684j
  3.61386763-3.79231684j  0.7744952 +0.j        ]

Eigenvectors (5, 5) :
 [[ 0.3801489 +0.j          0.69021428+0.j         -0.32692887+0.24693726j
  -0.32692887-0.24693726j  0.55407276+0.j        ]
 [ 0.34493793+0.j         -0.55055459+0.j         -0.03446564+0.24274051j
  -0.03446564-0.24274051j -0.27697669+0.j        ]
 [ 0.55423261+0.j          0.01530606+0.j          0.37586243-0.20380539j
   0.37586243+0.20380539j -0.76007966+0.j        ]
 [ 0.43210801+0.j         -0.46542057+0.j         -0.1654183 -0.50470226j
  -0.1654183 +0.50470226j -0.02520394+0.j        ]
 [ 0.49255813+0.j          0.06035982+0.j          0.5541948 +0.j
   0.5541948 -0.j          0.19475873+0.j        ]]


In [20]:


## c) Kiểm tra eigenvector đầu tiên theo dot và theo eigenvalue có bằng nhau?
##    Nếu bằng nhau thì tái tạo A từ các eigenvalues và eigenvectors
B = A.dot(vectors[:, 0])
print('\nMa trận B:\n', B.astype(int))

C = vectors[:, 0] * values[0]
print('\nMa trận C:\n', C.astype(int))

Q = vectors
L = diag(values)

print('\nTái tạo ma trận A:\n', Q.dot(L).dot(inv(Q)).astype(int))


Ma trận B:
 [12 11 18 14 16]

Ma trận C:
 [12 11 18 14 16]

Tái tạo ma trận A:
 [[ 6 10  0  8  2]
 [10  3  6  2  0]
 [10  9  6  5  7]
 [ 9  0  7  6  4]
 [ 9 10  4  3  7]]


c:\program files\python36\lib\site-packages\ipykernel_launcher.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  after removing the cwd from sys.path.
c:\program files\python36\lib\site-packages\ipykernel_launcher.py:7: ComplexWarning: Casting complex values to real discards the imaginary part
  import sys
c:\program files\python36\lib\site-packages\ipykernel_launcher.py:12: ComplexWarning: Casting complex values to real discards the imaginary part
  if sys.path[0] == '':


In [11]:
"""=============================================================================
Ex2: EIGENDECOMPOSITION
    Câu 2:
        a) Tạo ma trận A(4 x 4) chứa các giá trị ngẫu nhiên trong khoảng 1 - 9
        b) Tạo ma trận đối xứng B = A.A_T
        c) Phân tích eigenvalues và eigenvectors của B
============================================================================="""

import numpy as np
from numpy.linalg import eig

## a) Tạo ma trận A(5 x 5) chứa các giá trị ngẫu nhiên trong khoảng 1 - 10
m, n, min, max = 4, 4, 1, 9
A = np.random.randint(1, 10, (n, n))
print('Ma trận A', A.shape, ':\n', A)

## b) Tạo ma trận đối xứng B = A.A_T
B = A @ A.T
print('\nMa trận đối xứng B', B.shape, ':\n', B)

## c) Phân tích eigenvalues và eigenvectors
values, vectors = eig(B)
print('\nSố lượng giá trị riêng: %d' %len(values))
print('\nEigenvalues',  values.shape,  ':\n', values)
print('\nEigenvectors', vectors.shape, ':\n', vectors)



Ma trận A (4, 4) :
 [[5 7 8 5]
 [3 1 6 5]
 [3 2 6 8]
 [9 6 4 2]]

Ma trận đối xứng B (4, 4) :
 [[163  95 117 129]
 [ 95  71  87  67]
 [117  87 113  79]
 [129  67  79 137]]

Số lượng giá trị riêng: 4

Eigenvalues (4,) :
 [418.88151135  53.94843137   8.86078162   2.30927567]

Eigenvectors (4, 4) :
 [[-0.61266024 -0.07269221 -0.78686384  0.01444214]
 [-0.38276576  0.37480019  0.2485891  -0.80698119]
 [-0.473386    0.56764     0.3269519   0.58889131]
 [-0.50402728 -0.72940077  0.46059876  0.04218768]]
